In [2]:
# Imports

import numpy as np
import pandas as pd
from shared.utils import load_data
from datasets import preprocess_dataset, datasets_types
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.stats import beta as beta_dist
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, recall_score, roc_auc_score

seed = 42

In [3]:
load_dataset = False
name = "CIC-IDS_2019_MAB"
if not load_dataset:
    # Preprocesar el dataset
    """      """
    df = load_data(
        [
            "./shared/data/CIC_2019/DrDoS_MSSQL.csv"
        ],
        seed
    )
    print("Dataset cargado")
    df_preprocessed = preprocess_dataset(
        df, save=True, dataset_type="CIC_2019", seed=seed, load=load_dataset, name_save=name, name_load=name)
    print("Dataset Preprocesado")
else:
    df_preprocessed = preprocess_dataset(
        pd.DataFrame(), save=True, dataset_type="CIC_2019", seed=seed, load=load_dataset, name_save=name, name_load=name)
    print("Dataset Preprocesado")


FileNotFoundError: [Errno 2] No such file or directory: './shared/data/CIC_2019/DrDoS_MSSQL.csv'

In [23]:
class MultiArmedBanditThompsonSampling:

    def __init__(self, n_arms, n_clusters):
        self.n_arms = n_arms
        self.n_clusters = n_clusters
        self.arms = [RandomForestClassifier(), DecisionTreeClassifier(),
                     GaussianNB(), LogisticRegression(), mlp]
        self.cluster_centers = None
        self.cluster_assignments = None
        self.reward_sums = {}
        for cluster in range(n_clusters):
            self.reward_sums[cluster] = np.zeros(n_arms)
        self.alpha = np.ones(self.n_arms)
        self.beta = np.ones(self.n_arms)

    def train(self, X_train, y_train):
        kmeans = KMeans(n_clusters=self.n_clusters)
        self.cluster_assignments = kmeans.fit_predict(X_train)
        self.cluster_centers = kmeans.cluster_centers_
        # Print the number of samples in each cluster

        for i in range(self.n_clusters):
            print('Cluster {}: {}'.format(
                i, np.sum(self.cluster_assignments == i)))
            cluster_mask = self.cluster_assignments == i
            cluster_X_train = X_train[cluster_mask]
            cluster_y_train = y_train[cluster_mask]
            for arm in range(self.n_arms):
                print('Training arm {} on cluster {}'.format(arm, i))
                arm_mask = cluster_y_train == arm
                arm_X_train = cluster_X_train[arm_mask]
                arm_y_train = cluster_y_train[arm_mask]
                if len(arm_X_train) > 0 and len(np.unique(arm_y_train)) > 1:
                    self.arms[arm].fit(arm_X_train, arm_y_train)
                else:
                    self.arms[arm].fit(X_train, y_train)

        # Set the arms rewards for each cluster
        for i in range(self.n_clusters):
            cluster_mask = self.cluster_assignments == i
            cluster_X_test = X_train[cluster_mask]
            cluster_y_test = y_train[cluster_mask]
            for arm in range(self.n_arms):
                print('Setting reward_sums arm {} on cluster {}'.format(arm, i))
                arm_mask = cluster_y_test == arm
                arm_X_test = cluster_X_test[arm_mask]
                arm_y_test = cluster_y_test[arm_mask]
                if len(arm_X_test) > 0:
                    arm_y_pred = self.arms[arm].predict(arm_X_test)
                    self.reward_sums[i][arm] = np.mean(
                        arm_y_pred == arm_y_test)

    def select_arm(self, cluster):
        # Select the arm with the highest reward
        theta = np.zeros(self.n_arms)
        for arm in range(self.n_arms):
            theta[arm] = np.random.beta(self.alpha[arm] + self.reward_sums[cluster]
                                        [arm], self.beta[arm] + 1 - self.reward_sums[cluster][arm])
        return np.argmax(theta)

    def predict(self, X_test):
        # Select the arm for each sample
        arms = np.zeros(len(X_test))
        for i in range(len(X_test)):
            cluster = np.argmin(np.linalg.norm(
                self.cluster_centers - X_test[i], axis=1))
            arms[i] = self.select_arm(cluster)
        # Predict using the selected arm
        y_pred = np.zeros(len(X_test))
        for arm in range(self.n_arms):
            arm_mask = arms == arm
            arm_X_test = X_test[arm_mask]
            if len(arm_X_test) > 0:
                y_pred[arm_mask] = self.arms[arm].predict(arm_X_test)
        return y_pred, arms


In [24]:
# Train the MAB
mab = MultiArmedBanditThompsonSampling(n_arms=3, n_clusters=2)
mab.train(df_preprocessed.x_train, df_preprocessed.y_train)

Cluster 0: 2789828
Training arm 0 on cluster 0
Training arm 1 on cluster 0
Training arm 2 on cluster 0
Cluster 1: 377320
Training arm 0 on cluster 1
Training arm 1 on cluster 1
Training arm 2 on cluster 1
Setting reward_sums arm 0 on cluster 0
Setting reward_sums arm 1 on cluster 0
Setting reward_sums arm 2 on cluster 0
Setting reward_sums arm 0 on cluster 1
Setting reward_sums arm 1 on cluster 1
Setting reward_sums arm 2 on cluster 1


In [25]:
# Test the MAB
y_pred, selected_arms = mab.predict(df_preprocessed.x_test)
# Transform the y_pred values to 0 and 1 strings
y_test = np.array([int(y) for y in df_preprocessed.y_test])
y_pred = np.array([int(y) for y in y_pred])

# Print y_pred unique values
print(np.unique(y_pred))
# Print y_test unique values
print(np.unique(y_test))

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred, average='macro'))
print("F1 Score:", f1_score(y_test, y_pred, average='macro'))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))

[0 1]
[0 1]
              precision    recall  f1-score   support

           0       0.56      0.94      0.70       602
           1       1.00      1.00      1.00   1356748

    accuracy                           1.00   1357350
   macro avg       0.78      0.97      0.85   1357350
weighted avg       1.00      1.00      1.00   1357350

Accuracy: 0.9996507901425572
Recall: 0.9691080561338743
F1 Score: 0.852157034949286
ROC AUC Score: 0.9691080561338742
